In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import sys
sys.path.append('..')
from detector.input_pipeline import PoseResidualNetworkPipeline

In [ ]:
cmap = plt.cm.get_cmap('autumn')
new_cmap = cmap(np.arange(cmap.N))
new_cmap[:, -1] = np.sqrt(np.linspace(0, 1, cmap.N))  # set alpha
cmap = ListedColormap(new_cmap)  # create new colormap

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/home/dan/datasets/COCO/multiposenet/train/shard-0001.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0002.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0003.tfrecords',
]

with tf.device('/cpu:0'):
    pipeline = PoseResidualNetworkPipeline(files, is_training=True, batch_size=10)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()

# Show an image

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run([features, labels])

    heatmaps = output[0]
    binary_masks = output[1]

In [ ]:
i = 0

w, h = 36, 56
background = Image.new('RGBA', (w * 2, h * 17), (255, 255, 255, 255))
draw = ImageDraw.Draw(background, 'RGBA')
heatmaps = (cmap(heatmaps[i]) * 255).astype('uint8')
binary_masks = (binary_masks[i] * 255).astype('uint8')

for j in range(17):

    heat = Image.fromarray(heatmaps[:, :, j])
    mask = Image.fromarray(binary_masks[:, :, j])

    background.paste(heat, (0, j*h))
    background.paste(mask, (w, j*h))

w, h = background.size
background.resize((w * 5, h * 5))